In [1]:
# Algoritmo 1
# En este algoritmo voy a usar el indicador Tecnico de las nubes de Ichimoku
# La señal de compra viene definida por varias condiciones dentro del indicador,
# estas son que el precio y el tenkan esten por encima de los senkou y que el tenkan
# sea inferior al precio entre otras.
# A la hora de buscar salidas de las operaciones se opta por un SL inicial en el
# senkou y dos condicionantes de salida, aunque el principal es la pendiente
# negativa de una ema corta
# Respecto a la gestion de capital se ha optado por una asignacion fij
import requests
import json
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
from datetime import date
from api_bme import APIBMEHandler

In [2]:
# Descargo los datos de la api de BME
APIBME = APIBMEHandler('DAX', 'ignaciobermejo_algo1')
maestro_df = APIBME.get_ticker_master()
data_close, data_high, data_low, data_open, data_vol = APIBME.get_data()
benchmark = APIBME.get_close_data_ticker('benchmark')


In [3]:
# Defino los componentes que conforma Ichimoku que son: Tenkan Sen(linea de conversion), Kijun Sen (linea base), Chinkou Span (precio actual retardado), Senkou Span A(extension ppal A), Senkou Span B(extension ppal B), Kumo(nube ichimoku A-B)
def obv(data, volumen):
    vol=pd.DataFrame(index=data.index, columns=data.columns, data=0)
    vol[data.diff()>0]=volumen
    vol[data.diff()<0]= -(volumen)
    vol=cumsum()
    return vol

def ichimoku(high,low,close,periodo_1,periodo_2,preiodo_3):
    chinkou_span=close*1
    chinkou_span=chinkou_span.shift(periodo_2,freq='B')#tenia puesto tshift pero me salta que se elimina en futuras veriones
    tenkan_sen=(high.rolling(periodo_1).max()+low.rolling(periodo_1).min())/2
    kijun_sen=(high.rolling(periodo_2).max()+low.rolling(periodo_2).min())/2
    senkou_aux1=(tenkan_sen+kijun_sen)/2
    senkou_span_A=senkou_aux1.shift(periodo_2,freq='B')
    senkou_aux2=(high.rolling(preiodo_3).max()+low.rolling(preiodo_3).min())/2
    senkou_span_B=senkou_aux2.shift(periodo_2,freq='B')
    return senkou_span_A, senkou_span_B, chinkou_span, tenkan_sen, kijun_sen

def MME(data, periodo):
    mme=data.ewm(span=periodo,adjust=False).mean()
    return mme

In [4]:
#Calculo los retornos diarios
data_returns=np.log(data_close).diff()

#Aplico los componentes de ichimoku
periodo_1 = 18
periodo_2 = 52
periodo_3 = 104

senkou_span_A,senkou_span_B,chinkou_span,tenkan_sen,kijun_sen=ichimoku(data_high,data_low,data_close, periodo_1,periodo_2,periodo_3)
mme_18=MME(data_close,periodo_1)
mme_5=MME(data_close,5)

#Aplico los indicadores y selecciono las operaciones
posiciones=pd.DataFrame(index=data_close.index,columns=data_close.columns,data=0)
posiciones_aux=pd.DataFrame(index=data_close.index,columns=data_close.columns,data=0)

filtro_1=(data_close.iloc[periodo_2:,:]-senkou_span_A.iloc[:-periodo_2,:]>0) & (data_close.iloc[periodo_2:,:]-senkou_span_B.iloc[:-periodo_2,:]>0)
filtro_2 = (tenkan_sen.iloc[periodo_2:,:] - senkou_span_A.iloc[:-periodo_2,:]>0) & (tenkan_sen.iloc[periodo_2:,:] - senkou_span_B.iloc[:-periodo_2,:]>0)
filtro_3 = (data_low.iloc[periodo_2:,:] - senkou_span_A.iloc[:-periodo_2,:]>0) & (data_low.iloc[periodo_2:,:] - senkou_span_B.iloc[:-periodo_2,:]>0)
posiciones_aux[filtro_1] = 1
posiciones_aux[posiciones_aux.diff()!= 1] = 0
posiciones_aux = posiciones_aux.ffill(limit = 6)
filtro_4 = (tenkan_sen<data_close) & (posiciones_aux == 1) 
filtro_4 = ((tenkan_sen*1.005)<data_close)
posiciones[filtro_1 & filtro_2 & filtro_3 & filtro_4] = 1
comprar_posiciones = posiciones*1

p_compra = posiciones*data_close
p_compra[p_compra ==0]=None
p_compra = p_compra.ffill()
p_compra[data_close.isna()] = None

In [5]:
#Defino el Stop Loss
stop_loss=senkou_span_A*1
stop_loss[senkou_span_B>senkou_span_A]=senkou_span_B
stop_loss=stop_loss*0.997
stop_loss=comprar_posiciones*stop_loss
stop_loss[stop_loss==0]=None
stop_loss=stop_loss.ffill()
stop_loss[data_close>(1.015*p_compra)]=p_compra
stop_loss[data_close.isna()]=None

#Defino cuando vender las posiciones
posiciones[data_close-stop_loss<0]=-1
posiciones[(data_close<(tenkan_sen*0.99)) & (mme_18.diff()<0)]=-1
posiciones[mme_5.diff()<0]=-1

#Creo un DF de todas las compras y ventas
posiciones[posiciones==0]=None
posiciones=posiciones.ffill()
posiciones[posiciones==-1]=0
posiciones=posiciones.ffill()
posiciones[data_close.isna()]=None

In [6]:
#Defino cuanto comprar y pongo un volumne fijo de 0.03
vol_invertir=posiciones*0.03
vol_invertir[vol_invertir.isna()]=0

In [7]:
def gen_alloc_data(ticker, alloc):
    return {'ticker': ticker,
            'alloc': alloc}
tickers = data_close.columns
tickers = tickers.to_series()
alloc = vol_invertir.iloc[-1,:].values

hoy = date.today().strftime('%Y-%m-%d')

allocation = [gen_alloc_data(tickers[i], alloc[i]) for i in np.arange(0,data_close.shape[1])]
APIBME.post_alloc(hoy,allocation)

{'date': '2021-09-01', 'result': True}
